In [ ]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 10.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers --upgrade
!pip install datasets accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.5
    Uninstalling huggingface-hub-0.33.5:
      Successfully uninstalled huggingface-hub-0.33.5
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import  AutoTokenizer, AutoModelForSequenceClassification,  Trainer, TrainingArguments,  EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random
import os
import sys
import tempfile
import time

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

def set_seed(seed=1234):

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    os.environ['PYTHONHASHSEED'] = str(seed)



cuda


In [ ]:
import pandas as pd

NewsBias = pd.read_csv('/content/drive/MyDrive/Samsung/PROY/NewsBias.csv')
NewsBias.head()

,id,text,label
0,0,what got us through for many it was hobbies re...,leaning-left
1,1,new year s eve gatherings could accelerate ent...,leaning-left
2,2,entity traditions that you can still participa...,center
3,3,here is how countries around the world are rin...,leaning-right
4,4,police in entity said on thursday evening that...,left


In [ ]:
class BiasDataset:

  def __init__(self, texts, labels, ids, tokenizer, max_len=512, pad="max_length", trunc=True, rt='pt'):
    self.texts = list(texts)
    self.labels = labels
    self.ids = ids
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.pad = pad
    self.trunc = trunc
    self.rt = rt

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    inputs = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,padding=self.pad, truncation=self.trunc,
        return_tensors=self.rt
    )

    return {
        'input_ids': inputs['input_ids'].flatten(),
        'attention_mask': inputs['attention_mask'].flatten(),
        'labels': torch.tensor(self.labels[idx], dtype=torch.long),
        'id': torch.tensor(self.ids[idx], dtype=torch.long)
    }

In [ ]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def bias_classification(trainInfo,model_name= 'bert-base-uncased',metrics=compute_metrics,**args):

    le = LabelEncoder()
    labels = le.fit_transform(trainInfo['label'])
    nlabels = len(le.classes_)

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type="single_label_classification"
    )

    train_df, val_df = train_test_split(
        trainInfo, test_size=0.20, random_state=100, stratify=labels
    )

    y_train = le.transform(train_df['label'])
    y_val   = le.transform(val_df['label'])

    train_dataset = BiasDataset(
        train_df['text'].tolist(),
        y_train.tolist(),
        train_df['id'].astype(int).tolist(),
        tokenizer
    )
    val_dataset = BiasDataset(
        val_df['text'].tolist(),
        y_val.tolist(),
        val_df['id'].astype(int).tolist(),
        tokenizer
    )

    training_args = TrainingArguments(
        report_to="none",
        output_dir= args.get('output_dir', './results'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model','accuracy')
    )

    early_stop = EarlyStoppingCallback(
        early_stopping_patience=args.get('early_stopping_patience', 4)
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=metrics,
        callbacks=[early_stop]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    return model, eval_results


In [ ]:
set_seed()

modelname = 'bert-base-uncased'

params = {
    "num_train_epochs":           15,
    "learning_rate":              1e-5,
    "max_length":                 256,
}

time0 = time.time()

model, results = bias_classification(
    trainInfo       = NewsBias,
    model_name      = modelname,
    compute_metrics = compute_metrics,
    **params
)
time_en1 = time.time() - time0

print(f"Tiempo de entrenamiento: {time_en1:.1f}s")
print("Métricas de validación:", results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.276100,1.350562,0.435146,0.386947,0.446734,0.405882
2,1.103400,1.180339,0.516504,0.509000,0.578101,0.492455
3,1.009300,1.118354,0.544863,0.530815,0.562684,0.522454
4,0.853900,1.119298,0.570897,0.543699,0.583931,0.542595


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.276100,1.350562,0.435146,0.386947,0.446734,0.405882
2,1.103400,1.180339,0.516504,0.509000,0.578101,0.492455
3,1.009300,1.118354,0.544863,0.530815,0.562684,0.522454
4,0.853900,1.119298,0.570897,0.543699,0.583931,0.542595
5,0.854200,1.133776,0.590423,0.590310,0.602654,0.589500
6,0.667200,1.186642,0.587169,0.586630,0.605921,0.586165
7,0.412500,1.224520,0.608554,0.605020,0.613333,0.602992


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
